## Activity inside ROI

Calculates the ROI for an embryo, using the structural channel to calculate the mask.

This code relies on the naming convention in this codebase to work properly: each embryo should have a file ending in `ch1.tif` (active channel) and another one ending in `ch2.tif` (structural channel).

To speed up the analysis, the ROI can be cached with the function `roi.cache_rois`, and passed to `activity.plot_activity` through the argument `mask_path`.

In [1]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from tifffile import imread

from pasnascope import roi, activity

In [2]:
project_dir = Path(os.getcwd()).parent
img_dir = os.path.join(project_dir, 'data', 'embryos')

# All structural channel movies end with the suffix ch2
active = [f for f in sorted(os.listdir(img_dir)) if f.endswith('ch1.tif')]
struct = [f for f in sorted(os.listdir(img_dir)) if f.endswith('ch2.tif')]

### Percentual error caused by downsampling

Percentual error caused by downsampling. 
The error is the percentual difference between the average values for each image slice.

In [ ]:
i = 1
print(active[i])
active_img = imread(os.path.join(img_dir,active[i]))
struct_img = imread(os.path.join(img_dir,struct[i]))

mask = roi.get_roi(struct_img, window=50)
masked_activity_50 = activity.apply_mask(active_img, mask)
activity_window_50 = np.average(masked_activity_50, axis=(1,2))

mask = roi.get_roi(struct_img, window=10)
masked_activity_10 = activity.apply_mask(active_img, mask)
activity_window_10 = np.average(masked_activity_10, axis=(1,2))

mask = roi.get_roi(struct_img, window=5)
masked_window_5 = activity.apply_mask(active_img, mask)
activity_window_5 = np.average(masked_window_5, axis=(1,2))

mask = roi.get_roi(struct_img, window=1)
masked_window_1 = activity.apply_mask(active_img, mask)
activity_window_1 = np.average(masked_window_1, axis=(1,2))

diff_50 = (activity_window_1 - activity_window_50)/activity_window_1
diff_10 = (activity_window_1 - activity_window_10)/activity_window_1
diff_5 = (activity_window_1 - activity_window_5)/activity_window_1

fig, ax = plt.subplots()

ax.plot(diff_50[:-100], color='red', label='50:1', alpha=0.7, linewidth=1)
ax.plot(diff_10[:-100],  label='10:1', alpha=0.7, linewidth=1)
ax.plot(diff_5[:-100],  label='5:1', alpha=0.7, linewidth=1)
ax.legend()
fig.suptitle('Percentual error for downsampling')
plt.show()

Calculates `dff` for an image:

In [ ]:
i = 3
active_img = imread(os.path.join(img_dir,active[i]), key=range(1000))
struct_img = imread(os.path.join(img_dir,struct[i]), key=range(1000))

print(active[i])
mask = roi.get_roi(struct_img)

masked_active = activity.apply_mask(active_img, mask)
masked_struct = activity.apply_mask(struct_img, mask)

signal_active = activity.get_activity(masked_active)
signal_struct = activity.get_activity(masked_struct)

r_signal = activity.ratiometric_activity(signal_active, signal_struct)
baseline = activity.compute_baseline(r_signal)

dff = activity.get_dff(r_signal, baseline)

fig, ax = plt.subplots()
ax.plot(dff[:-100])
plt.show()
